# Adding and Removing Data
We will be working with the `data/earthquakes.csv` file again, so we need to handle our imports and read it in.

## About the Data
In this notebook, we will be working with Earthquake data from September 18, 2018 - October 13, 2018 (obtained from the US Geological Survey (USGS) using the [USGS API](https://earthquake.usgs.gov/fdsnws/event/1/))

## Setup

In [1]:
import numpy as np
import pandas as pd

df = pd.read_csv(
    'data/earthquakes.csv', 
    usecols=['time', 'title', 'place', 'magType', 'mag', 'alert', 'tsunami']
)

## Creating new data
### Adding new columns
New columns get added to the right of the dataframe and can be a single value:

In [2]:
df['ones'] = 1
df.head()

,alert,mag,magType,place,time,title,tsunami,ones
0,NaN,1.35,ml,"9km NE of Aguanga, CA",1539475168010,"M 1.4 - 9km NE of Aguanga, CA",0,1
1,NaN,1.29,ml,"9km NE of Aguanga, CA",1539475129610,"M 1.3 - 9km NE of Aguanga, CA",0,1
2,NaN,3.42,ml,"8km NE of Aguanga, CA",1539475062610,"M 3.4 - 8km NE of Aguanga, CA",0,1
3,NaN,0.44,ml,"9km NE of Aguanga, CA",1539474978070,"M 0.4 - 9km NE of Aguanga, CA",0,1
4,NaN,2.16,md,"10km NW of Avenal, CA",1539474716050,"M 2.2 - 10km NW of Avenal, CA",0,1


...or a Boolean mask:

In [3]:
df['mag_negative'] = df.mag < 0
df.head()

,alert,mag,magType,place,time,title,tsunami,ones,mag_negative
0,NaN,1.35,ml,"9km NE of Aguanga, CA",1539475168010,"M 1.4 - 9km NE of Aguanga, CA",0,1,False
1,NaN,1.29,ml,"9km NE of Aguanga, CA",1539475129610,"M 1.3 - 9km NE of Aguanga, CA",0,1,False
2,NaN,3.42,ml,"8km NE of Aguanga, CA",1539475062610,"M 3.4 - 8km NE of Aguanga, CA",0,1,False
3,NaN,0.44,ml,"9km NE of Aguanga, CA",1539474978070,"M 0.4 - 9km NE of Aguanga, CA",0,1,False
4,NaN,2.16,md,"10km NW of Avenal, CA",1539474716050,"M 2.2 - 10km NW of Avenal, CA",0,1,False


#### Adding the `parsed_place` column
We have an entity recognition problem on our hands with the `place` column. There are several entities that have multiple names in the data (e.g., CA and California, NV and Nevada).

In [4]:
df.place.str.extract(r', (.*$)')[0].sort_values().unique()

array(['Afghanistan', 'Alaska', 'Argentina', 'Arizona', 'Arkansas',
       'Australia', 'Azerbaijan', 'B.C., MX', 'Barbuda', 'Bolivia',
       'Bonaire, Saint Eustatius and Saba ', 'British Virgin Islands',
       'Burma', 'CA', 'California', 'Canada', 'Chile', 'China',
       'Christmas Island', 'Colombia', 'Colorado', 'Costa Rica',
       'Dominican Republic', 'East Timor', 'Ecuador', 'Ecuador region',
       'El Salvador', 'Fiji', 'Greece', 'Greenland', 'Guam', 'Guatemala',
       'Haiti', 'Hawaii', 'Honduras', 'Idaho', 'Illinois', 'India',
       'Indonesia', 'Iran', 'Iraq', 'Italy', 'Jamaica', 'Japan', 'Kansas',
       'Kentucky', 'Kyrgyzstan', 'Martinique', 'Mauritius', 'Mayotte',
       'Mexico', 'Missouri', 'Montana', 'NV', 'Nevada', 'New Caledonia',
       'New Hampshire', 'New Mexico', 'New Zealand', 'Nicaragua',
       'North Carolina', 'Northern Mariana Islands', 'Oklahoma', 'Oregon',
       'Pakistan', 'Papua New Guinea', 'Peru', 'Philippines',
       'Puerto Rico', 'Roman

Replace parts of the `place` names to fit our needs:

In [5]:
df['parsed_place'] = df.place.str.replace(
    r'.* of ', '' # remove anything saying <something> of <something>
).str.replace(
    r'the ', '' # remove things starting with "the"
).str.replace(
    r'CA$', 'California' # fix California
).str.replace(
    r'NV$', 'Nevada' # fix Nevada
).str.replace(
    r'MX$', 'Mexico' # fix Mexico
).str.replace(
    r' region$', '' # chop off endings with "region"
).str.replace(
    r'northern ', '' # remove "northern"
).str.replace(
    r'Fiji Islands', 'Fiji' # line up the Fiji places
).str.replace(
    r'^.*, ', '' # remove anything else extraneous from the beginning
).str.strip() # remove any extra spaces

Now we can use a single name to get all earthquakes for that place:

In [6]:
df.parsed_place.sort_values().unique()

array(['Afghanistan', 'Alaska', 'Argentina', 'Arizona', 'Arkansas',
       'Ascension Island', 'Australia', 'Azerbaijan', 'Balleny Islands',
       'Barbuda', 'Bolivia', 'British Virgin Islands', 'Burma',
       'California', 'Canada', 'Carlsberg Ridge',
       'Central East Pacific Rise', 'Central Mid-Atlantic Ridge', 'Chile',
       'China', 'Christmas Island', 'Colombia', 'Colorado', 'Costa Rica',
       'Dominican Republic', 'East Timor', 'Ecuador', 'El Salvador',
       'Fiji', 'Greece', 'Greenland', 'Guam', 'Guatemala', 'Haiti',
       'Hawaii', 'Honduras', 'Idaho', 'Illinois', 'India',
       'Indian Ocean Triple Junction', 'Indonesia', 'Iran', 'Iraq',
       'Italy', 'Jamaica', 'Japan', 'Kansas', 'Kentucky',
       'Kermadec Islands', 'Kuril Islands', 'Kyrgyzstan', 'Martinique',
       'Mauritius', 'Mayotte', 'Mexico', 'Mid-Indian Ridge', 'Missouri',
       'Montana', 'Nevada', 'New Caledonia', 'New Hampshire',
       'New Mexico', 'New Zealand', 'Nicaragua', 'North Carolina',


#### Using the `assign()` method to create columns
To create many columns at once or update existing columns, we can use `assign()`:

In [7]:
df.assign(
    in_ca=df.parsed_place.str.endswith('California'),
    in_alaska=df.parsed_place.str.endswith('Alaska')
).head()

,alert,mag,magType,place,time,title,tsunami,ones,mag_negative,parsed_place,in_ca,in_alaska
0,NaN,1.35,ml,"9km NE of Aguanga, CA",1539475168010,"M 1.4 - 9km NE of Aguanga, CA",0,1,False,California,True,False
1,NaN,1.29,ml,"9km NE of Aguanga, CA",1539475129610,"M 1.3 - 9km NE of Aguanga, CA",0,1,False,California,True,False
2,NaN,3.42,ml,"8km NE of Aguanga, CA",1539475062610,"M 3.4 - 8km NE of Aguanga, CA",0,1,False,California,True,False
3,NaN,0.44,ml,"9km NE of Aguanga, CA",1539474978070,"M 0.4 - 9km NE of Aguanga, CA",0,1,False,California,True,False
4,NaN,2.16,md,"10km NW of Avenal, CA",1539474716050,"M 2.2 - 10km NW of Avenal, CA",0,1,False,California,True,False


#### Concatenation
Say we were working with two separate dataframes, one with earthquakes accompanied by tsunamis and the other with earthquakes without tsunamis. If we wanted to look at earthquakes as a whole, we would want to concatenate the dataframes into a single one:

In [8]:
tsunami = df[df.tsunami == 1]
no_tsunami = df[df.tsunami == 0]

tsunami.shape, no_tsunami.shape

((61, 10), (9271, 10))

Concatenating along the row axis (`axis=0`) is equivalent to appending to the bottom. By concatenating our earthquakes with tsunamis and those without tsunamis, we get the full earthquake data set back:

In [9]:
pd.concat([tsunami, no_tsunami]).shape

(9332, 10)

Note that the previous result is equivalent to running the `append()` method of the dataframe:

In [10]:
tsunami.append(no_tsunami).shape

(9332, 10)

We have been working with a subset of the columns from the CSV file, but now we want to get some of the columns we ignored when we read in the data. Since we have added new columns in this notebook, we won't want to read in the file again and perform those operations again. Instead, we will concatenate along the columns (`axis=1`) to add back what we are missing:

In [11]:
additional_columns = pd.read_csv(
    'data/earthquakes.csv', usecols=['tz', 'felt', 'ids']
)
pd.concat(
    [df.head(2), additional_columns.head(2)], axis=1
)

,alert,mag,magType,place,time,title,tsunami,ones,mag_negative,parsed_place,felt,ids,tz
0,NaN,1.35,ml,"9km NE of Aguanga, CA",1539475168010,"M 1.4 - 9km NE of Aguanga, CA",0,1,False,California,NaN,",ci37389218,",-480.0
1,NaN,1.29,ml,"9km NE of Aguanga, CA",1539475129610,"M 1.3 - 9km NE of Aguanga, CA",0,1,False,California,NaN,",ci37389202,",-480.0


Notice what happens if the index doesn't align though:

In [12]:
additional_columns = pd.read_csv(
    'data/earthquakes.csv', usecols=['tz', 'felt', 'ids', 'time'], index_col='time'
)
pd.concat(
    [df.head(2), additional_columns.head(2)], axis=1
)

,alert,mag,magType,place,time,title,tsunami,ones,mag_negative,parsed_place,felt,ids,tz
0,NaN,1.35,ml,"9km NE of Aguanga, CA",1.539475e+12,"M 1.4 - 9km NE of Aguanga, CA",0.0,1.0,False,California,NaN,NaN,NaN
1,NaN,1.29,ml,"9km NE of Aguanga, CA",1.539475e+12,"M 1.3 - 9km NE of Aguanga, CA",0.0,1.0,False,California,NaN,NaN,NaN
1539475129610,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,",ci37389202,",-480.0
1539475168010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,",ci37389218,",-480.0


If the index doesn't align, we can align it before attempting the concatentation, which we will discuss in [chapter 3](https://github.com/stefmolin/Hands-On-Data-Analysis-with-Pandas/tree/master/ch_03):

Say we want to join the `tsunami` and `no_tsunami` dataframes, but the `no_tsunami` dataframe has an additional column. The `join` parameter specifies how to handle any overlap in column names (when appending to the bottom) or in row names (when concatenating to the left/right). By default, this is `outer`, so we keep everything; however, if we use `inner`, we will only keep what is in common:

In [13]:
pd.concat(
    [tsunami.head(2), no_tsunami.head(2).assign(type='earthquake')], join='inner'
)

,alert,mag,magType,place,time,title,tsunami,ones,mag_negative,parsed_place
36,NaN,5.00,mww,"165km NNW of Flying Fish Cove, Christmas Island",1539459504090,"M 5.0 - 165km NNW of Flying Fish Cove, Christm...",1,1,False,Christmas Island
118,green,6.70,mww,"262km NW of Ozernovskiy, Russia",1539429023560,"M 6.7 - 262km NW of Ozernovskiy, Russia",1,1,False,Russia
0,NaN,1.35,ml,"9km NE of Aguanga, CA",1539475168010,"M 1.4 - 9km NE of Aguanga, CA",0,1,False,California
1,NaN,1.29,ml,"9km NE of Aguanga, CA",1539475129610,"M 1.3 - 9km NE of Aguanga, CA",0,1,False,California


In addition, we use `ignore_index`, since the index doesn't mean anything for us here. This gives us sequential values instead of what we had in the previous result:

In [14]:
pd.concat(
    [tsunami.head(2), no_tsunami.head(2).assign(type='earthquake')], join='inner', ignore_index=True
)

,alert,mag,magType,place,time,title,tsunami,ones,mag_negative,parsed_place
0,NaN,5.00,mww,"165km NNW of Flying Fish Cove, Christmas Island",1539459504090,"M 5.0 - 165km NNW of Flying Fish Cove, Christm...",1,1,False,Christmas Island
1,green,6.70,mww,"262km NW of Ozernovskiy, Russia",1539429023560,"M 6.7 - 262km NW of Ozernovskiy, Russia",1,1,False,Russia
2,NaN,1.35,ml,"9km NE of Aguanga, CA",1539475168010,"M 1.4 - 9km NE of Aguanga, CA",0,1,False,California
3,NaN,1.29,ml,"9km NE of Aguanga, CA",1539475129610,"M 1.3 - 9km NE of Aguanga, CA",0,1,False,California


## Deleting Unwanted Data
Columns can be deleted using dictionary syntax with `del`:

In [15]:
del df['ones']
df.columns

Index(['alert', 'mag', 'magType', 'place', 'time', 'title', 'tsunami',
       'mag_negative', 'parsed_place'],
      dtype='object')

If we don't know if the column exists, we can use a `try`/`except` block:

In [16]:
try:
    del df['ones']
except KeyError:
    # handle the error here
    print('not there anymore')

not there anymore


We can also use `pop()`. This will allow us to use the series we remove later. Note there will be an error if the key doesn't exist, so we can also use a `try`/`except` here:

In [17]:
mag_negative = df.pop('mag_negative')
df.columns

Index(['alert', 'mag', 'magType', 'place', 'time', 'title', 'tsunami',
       'parsed_place'],
      dtype='object')

Notice we have a mask in `mag_negative` now:

In [18]:
mag_negative.value_counts()

False    8841
True      491
Name: mag_negative, dtype: int64

Now we can use `mag_negative` to select:

In [19]:
df[mag_negative].head()

,alert,mag,magType,place,time,title,tsunami,parsed_place
39,NaN,-0.10,ml,"6km NW of Lemmon Valley, Nevada",1539458844506,"M -0.1 - 6km NW of Lemmon Valley, Nevada",0,Nevada
49,NaN,-0.10,ml,"6km NW of Lemmon Valley, Nevada",1539455017464,"M -0.1 - 6km NW of Lemmon Valley, Nevada",0,Nevada
135,NaN,-0.40,ml,"10km SSE of Beatty, Nevada",1539422175717,"M -0.4 - 10km SSE of Beatty, Nevada",0,Nevada
161,NaN,-0.02,md,"20km SSE of Ronan, Montana",1539412475360,"M -0.0 - 20km SSE of Ronan, Montana",0,Montana
198,NaN,-0.20,ml,"60km N of Pahrump, Nevada",1539398340822,"M -0.2 - 60km N of Pahrump, Nevada",0,Nevada


### Using the `drop()` method
We can drop rows by passing a list of indices to the `drop()` method. Notice in the following example that when asking for the first 2 rows with `head()` we get the 3rd and 4th rows because we dropped the original first 2 with `drop([0, 1])`:

In [20]:
df.drop([0, 1]).head(2)

,alert,mag,magType,place,time,title,tsunami,parsed_place
2,NaN,3.42,ml,"8km NE of Aguanga, CA",1539475062610,"M 3.4 - 8km NE of Aguanga, CA",0,California
3,NaN,0.44,ml,"9km NE of Aguanga, CA",1539474978070,"M 0.4 - 9km NE of Aguanga, CA",0,California


The `drop()` method drops along the row axis by default. If we pass in `axis=1` for the column axis, we can delete columns:

In [21]:
df.drop(
    columns=[col for col in df.columns \
             if col not in \
             ['alert', 'mag', 'title', 'time', 'tsunami']]
).head()

,alert,mag,time,title,tsunami
0,NaN,1.35,1539475168010,"M 1.4 - 9km NE of Aguanga, CA",0
1,NaN,1.29,1539475129610,"M 1.3 - 9km NE of Aguanga, CA",0
2,NaN,3.42,1539475062610,"M 3.4 - 8km NE of Aguanga, CA",0
3,NaN,0.44,1539474978070,"M 0.4 - 9km NE of Aguanga, CA",0
4,NaN,2.16,1539474716050,"M 2.2 - 10km NW of Avenal, CA",0


We also have the option of passing the column names to the `columns` parameter instead of using `axis=1`:

In [22]:
df.drop(
    columns=[col for col in df.columns \
             if col not in \
             ['alert', 'mag', 'title', 'time', 'tsunami']]
).equals(
    df.drop(
        [col for col in df.columns \
         if col not in ['alert', 'mag', 'title', 'time', 'tsunami']],
        axis=1
    )
)

True

By default, `drop()`, along with the majority of `DataFrame` methods, will return a new `DataFrame` object. If we just want to change the one we are working with, we can pass `inplace=True`. This should be used with care:

In [23]:
df.drop(
    columns=[col for col in df.columns \
             if col not in \
             ['alert', 'mag', 'parsed_place', 'time', 'tsunami']],
    inplace=True
)
df.head()

,alert,mag,time,tsunami,parsed_place
0,NaN,1.35,1539475168010,0,California
1,NaN,1.29,1539475129610,0,California
2,NaN,3.42,1539475062610,0,California
3,NaN,0.44,1539474978070,0,California
4,NaN,2.16,1539474716050,0,California
